# Ch2

## CSV

CSV files are text, and can be loaded manually, as shown below, into dictionaries, or Pandas can be used.

In [21]:
import bz2
import csv
from collections import namedtuple
from datetime import datetime
import os
import pandas as pd

In [2]:
Column = namedtuple('Column', 'src dest convert')

In [4]:
def parse_timestamp(text):
    return datetime.strptime(text, '%Y-%m-%d %H:%M:%S')

In [5]:
columns = [
    Column('VendorID', 'vendor_id', int),
    Column('passenger_count', 'num_passengers', int),
    Column('tip_amount', 'tip', float),
    Column('total_amount', 'price', float),
    Column('tpep_dropoff_datetime', 'dropoff_time', parse_timestamp),
    Column('tpep_pickup_datetime', 'pickup_time', parse_timestamp),
    Column('trip_distance', 'distance', float),
]

In [10]:
def iter_records(file_name):
    with bz2.open(file_name, 'rt') as fp:
        reader = csv.DictReader(fp)
        for csv_record in reader:
            record = {}
            for col in columns:
                value = csv_record[col.src]
                record[col.dest] = col.convert(value)
            yield record


def example():
    from pprint import pprint
    filepath = os.path.join(path, file)
    for i, record in enumerate(iter_records(filepath)):
        if i >= 10:
            break
        pprint(record)

In [19]:
example()

{'distance': 2.57,
 'dropoff_time': datetime.datetime(2018, 11, 1, 6, 43, 24),
 'num_passengers': 1,
 'pickup_time': datetime.datetime(2018, 10, 31, 7, 10, 55),
 'price': 20.54,
 'tip': 4.74,
 'vendor_id': 2}
{'distance': 3.58,
 'dropoff_time': datetime.datetime(2018, 10, 31, 16, 50, 10),
 'num_passengers': 5,
 'pickup_time': datetime.datetime(2018, 10, 31, 16, 38, 25),
 'price': 13.8,
 'tip': 0.0,
 'vendor_id': 2}
{'distance': 2.39,
 'dropoff_time': datetime.datetime(2018, 10, 31, 20, 31, 47),
 'num_passengers': 1,
 'pickup_time': datetime.datetime(2018, 10, 31, 20, 23, 41),
 'price': 11.3,
 'tip': 1.0,
 'vendor_id': 2}
{'distance': 0.5,
 'dropoff_time': datetime.datetime(2018, 10, 31, 22, 48, 28),
 'num_passengers': 1,
 'pickup_time': datetime.datetime(2018, 10, 31, 22, 44, 24),
 'price': 5.8,
 'tip': 0.0,
 'vendor_id': 2}
{'distance': 1.81,
 'dropoff_time': datetime.datetime(2018, 10, 31, 23, 35, 30),
 'num_passengers': 1,
 'pickup_time': datetime.datetime(2018, 10, 31, 23, 22, 18),

In [18]:
path = './Ex_Files_Data_Ingestion_Python/Exercise Files/Ch02/02_01'
file = 'taxi.csv.bz2'

In [22]:
df = pd.read_csv(os.path.join(path, file))

In [24]:
df.dtypes

VendorID                   int64
tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count            int64
trip_distance            float64
RatecodeID                 int64
store_and_fwd_flag        object
PULocationID               int64
DOLocationID               int64
payment_type               int64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
dtype: object

In [25]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2018-10-31 07:10:55,2018-11-01 06:43:24,1,2.57,1,N,211,48,1,14.5,0.5,0.5,4.74,0.0,0.3,20.54
1,2,2018-10-31 16:38:25,2018-10-31 16:50:10,5,3.58,1,N,237,144,2,12.5,0.5,0.5,0.00,0.0,0.3,13.80
2,2,2018-10-31 20:23:41,2018-10-31 20:31:47,1,2.39,1,N,163,107,1,9.0,0.5,0.5,1.00,0.0,0.3,11.30
3,2,2018-10-31 22:44:24,2018-10-31 22:48:28,1,0.50,1,N,246,246,2,4.5,0.5,0.5,0.00,0.0,0.3,5.80
4,2,2018-10-31 23:22:18,2018-10-31 23:35:30,1,1.81,1,N,79,90,1,10.0,0.5,0.5,2.26,0.0,0.3,13.56


We can see that Pandas has stored the pickup_datetime and dropoff_datetime columns as objects. We need to tell Pandas that we want datetimes there.

In [26]:
time_cols = ['tpep_pickup_datetime', 'tpep_dropoff_datetime']

In [27]:
df = pd.read_csv(os.path.join(path, file), parse_dates=time_cols)

In [28]:
df.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                   int64
trip_distance                   float64
RatecodeID                        int64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
dtype: object

By default, Pandas loads the entire dataset into memory. To prevent this, we can use the chunksize argument.

In [29]:
df = pd.read_csv(os.path.join(path, file), parse_dates=time_cols, chunksize=1000)

In [30]:
df

Now we can see that df is a text file reader. We can iterate over it to get the data.

In [31]:
for sub_df in df:
    print(len(sub_df))

1000
1000
1000
1000
1000
1000
1000
1000
1000
1000


## XML

XML, like CSV, is also text, and must be loaded manually.

In [32]:
import bz2
import xml.etree.ElementTree as xml

import pandas as pd

# Data conversions
conversion = [
    ('vendor', int),
    ('people', int),
    ('tip', float),
    ('price', float),
    ('pickup', pd.to_datetime),
    ('dropoff', pd.to_datetime),
    ('distance', float),
]


def iter_rides(file_name):
    with bz2.open(file_name, 'rt') as fp:
        tree = xml.parse(fp)

    rides = tree.getroot()
    for elem in rides:
        record = {}
        for tag, func in conversion:
            text = elem.find(tag).text
            record[tag] = func(text)
        yield record


def load_xml(file_name):
    records = iter_rides(file_name)
    return pd.DataFrame.from_records(records)

In [34]:
path = './Ex_Files_Data_Ingestion_Python/Exercise Files/Ch02/02_02'
file = 'taxi.xml.bz2'
df = load_xml(os.path.join(path, file))
print(df.dtypes)
print(df.head())

vendor               int64
people               int64
tip                float64
price              float64
pickup      datetime64[ns]
dropoff     datetime64[ns]
distance           float64
dtype: object
   vendor  people   tip  price              pickup             dropoff  \
0       2       1  4.74  20.54 2018-10-31 07:10:55 2018-11-01 06:43:24   
1       2       5  0.00  13.80 2018-10-31 16:38:25 2018-10-31 16:50:10   
2       2       1  1.00  11.30 2018-10-31 20:23:41 2018-10-31 20:31:47   
3       2       1  0.00   5.80 2018-10-31 22:44:24 2018-10-31 22:48:28   
4       2       1  2.26  13.56 2018-10-31 23:22:18 2018-10-31 23:35:30   

   distance  
0      2.57  
1      3.58  
2      2.39  
3      0.50  
4      1.81  


## JSON

Below is code to read the JSON file in section 02_05. First, it has to parse the date and time string into a datetime object.

In [35]:
path = './Ex_Files_Data_Ingestion_Python/Exercise Files/Ch02/02_05'
file = 'taxi.jl'

In [36]:
import json
from datetime import datetime, timedelta


def parse_time(ts):
    """
    >>> parse_time('2018-10-31T07:10:55.000Z')
    datetime.datetime(2018, 10, 31, 7, 10, 55)
    """
    # [:-1] trims Z suffix
    return datetime.fromisoformat(ts[:-1])


def fix_pair(pair):
    key, value = pair
    if key not in ('pickup', 'dropoff'):
        return pair
    return key, parse_time(value)


def pairs_hook(pairs):
    return dict(fix_pair(pair) for pair in pairs)


durations = []
with open(os.path.join(path, file)) as fp:
    for line in fp:
        obj = json.loads(line, object_pairs_hook=pairs_hook)
        duration = obj['dropoff'] - obj['pickup']
        durations.append(duration)

avg_duration = sum(durations, timedelta()) / len(durations)
print(f'average ride duration: {avg_duration}')

average ride duration: 0:17:28.555600
